Etapas



1.   Transformas as coordenadas geodesicas em cartesianas
2.   Utilizar a formula dos tres parametros para calcular as coordenadas cartesianas em outro sistema
2.   Transformas as coordenadas cartesianas em geodésicas





In [1]:
# Função para transformar a coordenada GMS para GD

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

In [2]:
dms2dd(degrees, minutes, seconds, direction)

NameError: name 'degrees' is not defined

In [3]:
import math
import numpy as np

def geod2cart(a1,f1,latGD,longGD,h):

  # Parâmetros do Sistema 1

  # Semi-eixo menor (b1)
  b1=a1*(1-f1)
  #print(round(b1,3),"b1")

  # 1º excentricidade ao quadrado (e2_1)
  e2_1=2*f1-f1**2
  #print(round(e2_1,10),"e2_1")

  # Grande Normal
  N1=(a1/math.sqrt(1-e2_1*((np.sin(np.radians(latGD))**2))))
  #print(round(N1,3),"Grande Normal no S1")

  # Segunda Excentricidade ao quadrado no S1 (e2´_1)
  ee2_1= e2_1/(1-e2_1)
  #print(round(ee2_1,10),"Segunda Excentricidade ao quadrado no S1 (e2´_1)")

  # Cálculo das coordenadas Cartesianas no SAD69
  X1=(N1+h)*np.cos(np.radians(latGD))*np.cos(np.radians(longGD))
  #print(round(X1,3),"X1")

  Y1=(N1+h)*np.cos(np.radians(latGD))*np.sin(np.radians(longGD))
  #print(round(Y1,3),"Y1")

  Z1=((N1*(1-e2_1))+h)*np.sin(np.radians(latGD))
  #print(round(Z1,3),"Z1")
  return [X1,Y1,Z1]


In [4]:
geod2cart(6378160, 1/298.25, dms2dd(22, 31, 14.1362,"S"), dms2dd(49, 20, 31.4116,"W"), 815.34)[0]

3841113.488813773

In [5]:
# Função para calcular as coordenadas cartesianas no s1 para o s2

def CartS1_CartS2(X1Y1Z1:list,delX12,delY12,delZ12):

  X2=X1Y1Z1[0]+delX12
  Y2=X1Y1Z1[1]+delY12
  Z2=X1Y1Z1[2]+delZ12

  return [X2,Y2,Z2]    

In [6]:
CartS1_CartS2(geod2cart(6378160, 1/298.25, dms2dd(22, 31, 14.1362,"S"), dms2dd(49, 20, 31.4116,"W"), 815.34),-67.35,3.88,38.22)

[3841046.1388137727, -4472338.983769619, -2428046.142908231]

In [7]:
def cart2geod(XYZ:list,a,f,b):

  # Cálculo de p
  p=math.sqrt(XYZ[0]**2+XYZ[1]**2)
  print(p,"p")


  # Cálculo da tangente de U
  tgU=(XYZ[2]*a)/(p*b)
  print(tgU,"tgU")

  # Encontrando o valor do ângulo U em graus decimais
  U=np.rad2deg(np.arctan(tgU))
  print(U,"U em graus decimais") 

  # Cálculo da primeira excentricidade ao quadrado
  e2_1=2*f-f**2
  print(e2_1,"e2_1")  

  # Cálculo da segunda excentricidade ao quadrado
  ee2_1=(e2_1/(1-e2_1))
  print(ee2_1,"ee2_1")

  # Latitude
  latGD=np.rad2deg(np.arctan((XYZ[2]+(ee2_1*b*(np.sin(np.radians(U))**3)))/(p-(e2_1*a*(np.cos(np.radians(U))**3)))))   
  print(latGD,"latGD")  

  # Longitude
  longGD=np.rad2deg(np.arctan(XYZ[1]/XYZ[0]))
  print(longGD,"longGD")

  # Cálculo da grande Normal N
  N1=(a/math.sqrt(1-e2_1*((np.sin(np.radians(latGD))**2))))
  print(round(N1,3),"Grande Normal no S1")

  # Altitude geometrica
  h=(p/(np.cos(np.radians(latGD))-N1)) 
  print(round(h,6),"Altitude geometrica")

  return [dd2dms(latGD),dd2dms(longGD),h]


In [9]:
cart2geod(CartS1_CartS2(geod2cart(6378160, 1/298.25, dms2dd(22, 31, 14.1362,"S"), dms2dd(49, 20, 31.4116,"W"), 815.34),-67.35,3.88,38.22),6378137,((6378137-6356752.31414)/6378137),6356752.31414)[2]

5895375.427081948 p
-0.413241588834423 tgU
-22.45244929778779 U em graus decimais
0.006694380023011879 e2_1
0.006739496775591552 ee2_1
-22.52043274971581 latGD
-49.342530765718735 longGD
6381271.159 Grande Normal no S1
-0.923856 Altitude geometrica


-0.923856099172895